In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import pickle
import re
import os

from pandas_profiling import ProfileReport
from pandas.tseries.offsets import DateOffset


from tqdm.autonotebook import tqdm
tqdm.pandas()

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'


In [ ]:
# definitions 

os.chdir('/Users/dp/Nova/OneDrive - NOVAIMS/1stSemester/DM/DMProject')
os.getcwd()

computed_data_path = 'computed_data/'
explorations_data_path = 'explorations/'

paths = [computed_data_path, explorations_data_path]
for path in paths:
    if not os.path.exists(path): 
        os.makedirs(path)


In [ ]:
# read donors raw
if 'donors.pickle' in os.listdir(computed_data_path): 
    with open(os.path.join(computed_data_path, 'donors.pickle'), 'rb') as f: 
        donors = pickle.load(f)
else: 
    donors = pd.read_csv('data/donors.csv')
    print('donors.head: \n', donors.head())

    with open(os.path.join(computed_data_path, 'donors.pickle'), 'wb') as f: 
        pickle.dump(donors, f)

In [ ]:
if all([f in os.listdir('computed_data/') 
        for f in ['history_feat_raw.pickle', 'cols_with_hist.pickle']
       ]):
    with open(os.path.join(computed_data_path, 'history_raw.pickle'), 'rb') as f: 
        history = pickle.load(f)
    print(history)

    with open(os.path.join(computed_data_path, 'cols_with_hist.pickle'), 'rb') as f: 
        cols_with_hist = pickle.load(f)
    #print(cols_with_hist)
else: 
    # cols with hist: wide to long 

    patterns = {'adate':'adate_', 'rfa':'rfa_\d{1,2}$', 'rdate':'RDATE_', 'ramnt':'RAMNT_'}
    cols_with_hist = []

    history = pd.DataFrame(columns=['CONTROLN', 'promo_no'])

    for p_name, pattern_base in patterns.items():

        pattern = '|'.join(['CONTROLN', pattern_base])
        cols_with_pattern = donors.columns[donors.columns.str.contains(pattern, case=False)]
        cols_with_hist.extend(cols_with_pattern)

        var_name = p_name+'_no'
        pattern_short = re.search('^.*_', pattern_base).group(0)
        tmp = pd.melt(donors[cols_with_pattern].copy(deep=True), id_vars='CONTROLN', var_name=var_name, value_name=p_name)
        #print(pattern_short)

        tmp.loc[tmp[p_name] == ' ', p_name] = np.nan 
        tmp.dropna(subset=[p_name], inplace=True)


        tmp[var_name] = tmp[var_name].str.replace(pattern_short, '', case=False)
        tmp['promo_no'] = tmp[var_name]
        tmp.drop(columns=[var_name], inplace=True)
        #tmp['adate'] = pd.to_datetime(tmp.adate)
        #print(tmp)
        history = history.merge(tmp, on=['CONTROLN', 'promo_no'], how='outer')
        #print(p_name, cols_with_pattern)

    cols_with_hist.remove('CONTROLN')


    # cast to datetime
    history.loc[:, ['adate', 'rdate']] = history.loc[:, ['adate', 'rdate']].apply(pd.to_datetime, format='%Y-%m-%d', errors='raise')
    history.dtypes

    ## save history to file 

    with open(os.path.join(computed_data_path, 'history_raw.pickle'), 'wb') as f: 
        pickle.dump(history, f)

    with open(os.path.join(computed_data_path, 'cols_with_hist.pickle'), 'wb') as f: 
        pickle.dump(cols_with_hist, f)


In [ ]:
history[~(history.adate.isna()) & ~(history.ramnt.isna())]

In [ ]:
donors_with_history = history.CONTROLN.nunique() / donors.CONTROLN.nunique()
if donors_with_history == 1: 
    print('All donors have history')

# Some basic explorations: 

In [ ]:
# check missing values
print('NAs:\n',history.isna().sum())


# One time donors
# Have no rfa...

# check one example 
donors.loc[donors.CONTROLN == 3710,:]

# identifie multiple donors
donationsPerDoner = history.groupby('CONTROLN')['ramnt'].count() 
multiple_donors = donationsPerDoner[donationsPerDoner > 1].index.to_list()
multiple_donors


check = history.loc[(history.rfa.isna()) & (history.CONTROLN.isin(multiple_donors)), :]

# identifie multiple donors
checkPerDoner = check.groupby('CONTROLN')['ramnt'].count() 
multiple_donors_check = checkPerDoner[checkPerDoner > 1].index.to_list()
multiple_donors_check

check = check.loc[history.CONTROLN.isin(multiple_donors_check), :]
check = check.sort_values(['CONTROLN'])
check.head(10)

# Data Inconsistencies and Outliers

## Inconsistencies

In [ ]:
class Outliers(): 
    def __init__(self) -> pd.DataFrame:
        self.df = pd.DataFrame({
            'variable':pd.Series([], dtype='str'),
            'rule':pd.Series([], dtype='str'),
            'perc_loss_donors':pd.Series([], dtype='float'), 
            'perc_loss_obs':pd.Series([], dtype='float')
        })
    
    def append(self, var:str, rule:str, perc_loss_donors:float, perc_loss_obs:float):
        assert type(var) is str, 'var must be of type str'
        assert type(rule) is str, 'rule must be of type str'
        assert type(perc_loss_donors) in (float, int), 'diff must be of type float or int'
        assert type(perc_loss_obs) in (float, int), 'diff must be of type float or int'

        tmp = pd.DataFrame([{
            'variable': var,
            'rule':rule, 
            'perc_loss_donors': round(perc_loss_donors, 3), 
            'perc_loss_obs': round(perc_loss_obs, 3), 
        }])
        self.df = self.df.append(tmp, ignore_index=True)
        
    def drop(self, positions: list()): 
        self.df = self.df.drop(positions)
        
    def drop_all(self): 
        self.df = self.df.iloc[0:0]
        
        
def def_outliers_iqr(s: pd.Series, factor=1.5, omit_na=False) -> pd.Series: 
    q25 = s.quantile(.25)
    q75 = s.quantile(.75)
    iqr = (q75 - q25)

    upper_lim = q75 + factor * iqr
    lower_lim = q25 - factor * iqr
    
    if omit_na: 
        iqr_filter = ((s.between(lower_lim, upper_lim, inclusive=True)))

    else: 
        iqr_filter = ((s.between(lower_lim, upper_lim, inclusive=True)) | (s.isna()))
    
    return iqr_filter


def calc_perc_diff(df, ref): 
    return(round(((df.shape[0]-ref.shape[0]) / ref.shape[0]), 2))


def calc_perc_diff_num(x, ref): 
    return(round(((x-ref) / ref), 2))

def perc_loss_donors(history, filter_def): 
    return (1-history.loc[filter_def].CONTROLN.nunique() / history.CONTROLN.nunique())*100

def perc_loss_obs(history, filter_def): 
    return (1-history[filter_def].shape[0] / history.shape[0])*100
    


incons = Outliers()
outliers = Outliers()


### reply time

In [ ]:
# check reply times
replytime = (history.rdate - history.adate).dt.days
p = replytime.value_counts()
#sns.histplot(replytime)
# sns.histplot(p)

print(f'There are {(replytime < 0).sum()} obs with "adate" more recent then corresponing "rdate".')

# remove all doners where reply times are neg
replytime_filter_obs = (history.adate <= history.rdate)| history.rdate.isna() | history.adate.isna()
CONTROLN_with_rdate_adate_failure = history.loc[~replytime_filter_obs,:].CONTROLN.unique()
CONTROLN_with_rdate_adate_failure

replytime_filter = ~history.CONTROLN.isin(CONTROLN_with_rdate_adate_failure)

incons.append(
    var='rdate-adate', 
    rule='obs where rdate-adate >=0', 
    perc_loss_donors=perc_loss_donors(history, replytime_filter_obs), 
    perc_loss_obs=perc_loss_obs(history, replytime_filter_obs)
)
incons.df

incons.append(
    var='rdate-adate', 
    rule='all doners with at least one obs where rdate-adate >=0', 
    perc_loss_donors=perc_loss_donors(history, replytime_filter), 
    perc_loss_obs=perc_loss_obs(history, replytime_filter)
)
incons.df


In [ ]:
history.loc[(replytime_filter) & (history.CONTROLN == 1688)]

In [ ]:
history.loc[(replytime_filter_obs) & (history.CONTROLN == CONTROLN_with_rdate_adate_failure[0])]

### apply incons filter

In [ ]:
history = history.loc[replytime_filter]
history

## Outliers

### rdate

In [ ]:
from matplotlib.dates import DateFormatter
fig, ax = plt.subplots(nrows=2, sharey=False) # , figsize=(15,10)

sns.histplot(data=history, x='rdate', ax=ax[0])
date_form = DateFormatter("%Y-%m")
ax[0].xaxis.set_major_formatter(date_form)
ax[0].tick_params('x', labelrotation=30)

ax[0].title.set_text('Before')


# zoom in right tail
# sns.histplot(data=history.loc[history.rdate.dt.year == 2016,:], x='rdate')
# sns.histplot(data=history.loc[(history.rdate.dt.year == 2016) & (history.rdate.dt.month.isin([2,3,4])),:], x='rdate')

# zoom in left tail 
# sns.histplot(data=history.loc[history.rdate <= '2014-07-01',:], x='rdate')
# sns.histplot(data=history.loc[(history.rdate.dt.year == 2014) & (history.rdate.dt.month.isin([5,6,7])),:], x='rdate')



sns.histplot(data=history.loc[(history.rdate >= '2014-06-01') & (history.rdate <= '2016-03-01'),:], x='rdate', ax=ax[1])
ax[1].title.set_text('After')
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig(os.path.join(explorations_data_path, 'outliers_rdate.png'), dpi=200)

plt.show()


# remove rdate outliers and assign to new var 'hist' 
# remove all obs for respective doner
filter_rdate_obs = ((history.rdate.between('2014-06-01', '2016-03-01', inclusive=True)) | (history.rdate.isna()))

CONTROLN_with_rdate_outliers = history.loc[~filter_rdate_obs,:].CONTROLN.unique()

filter_rdate = ~history.CONTROLN.isin(CONTROLN_with_rdate_outliers)

outliers.append(
    var='rdate', 
    rule="obs where (history.rdate >= '2014-06-01') & (history.rdate <= '2016-03-01')",
    perc_loss_donors=perc_loss_donors(history, filter_rdate_obs), 
    perc_loss_obs=perc_loss_obs(history, filter_rdate_obs)
)
outliers.df

outliers.append(
    var='rdate', 
    rule="all donors with at least one obs where (history.rdate >= '2014-06-01') & (history.rdate <= '2016-03-01')",
    perc_loss_donors=perc_loss_donors(history, filter_rdate), 
    perc_loss_obs=perc_loss_obs(history, filter_rdate)
)
outliers.df

In [ ]:
history[filter_rdate_obs].rdate

In [ ]:
history[~filter_rdate_obs].rdate

In [ ]:
history.loc[(filter_rdate_obs) & (history.CONTROLN == 1688)]

In [ ]:
history.loc[(filter_rdate_obs) & (history.CONTROLN == CONTROLN_with_rdate_outliers[0])]

### ramnt

In [ ]:
# init plot
fig, ax = plt.subplots(nrows=2) # , figsize=(15,10)

# plot before
sns.histplot(data=history, x = 'ramnt', ax=ax[0])

#define filter  
filter_ramnt_obs = def_outliers_iqr(history.ramnt, factor=4, omit_na=False)

# plot after filter
sns.histplot(data=history.loc[filter_ramnt_obs,:], x = 'ramnt', ax=ax[1])

ax[0].title.set_text('Before')
ax[1].title.set_text('After')
#plt.xticks(rotation=30)
plt.tight_layout()

plt.savefig(os.path.join(explorations_data_path, 'outliers_ramnt.png'), dpi=200)
plt.show()


#append to oultier overview
outliers.append(
    var='ramnt', 
    rule='obs within iqr with factor 4',
    perc_loss_donors=perc_loss_donors(history, filter_ramnt_obs), 
    perc_loss_obs=perc_loss_obs(history, filter_ramnt_obs)
)
outliers.df


CONTROLN_with_ramnt_outliers = history.loc[~filter_ramnt_obs,:].CONTROLN.unique()
filter_ramnt = ~history.CONTROLN.isin(CONTROLN_with_ramnt_outliers)

#append to oultier overview
outliers.append(
    var='ramnt', 
    rule='all doners with at least one obs where iqr with factor 4',
    perc_loss_donors=perc_loss_donors(history, filter_ramnt), 
    perc_loss_obs=perc_loss_obs(history, filter_ramnt)
)
outliers.df


In [ ]:
tmp = history[filter_ramnt]
sns.histplot(data = tmp, x='ramnt')

In [ ]:
history.loc[(filter_ramnt) & (history.CONTROLN == 1688)]

In [ ]:
history.loc[(filter_ramnt_obs) & (history.CONTROLN == CONTROLN_with_ramnt_outliers[0])]

### Apply outlier filters

In [ ]:

# define filters

filters_outliers_obs = (
    filter_rdate_obs
    & 
    filter_ramnt_obs
)

# append to outliers overview
#append to oultier overview
outliers.append(
    var='total', 
    rule='only obs',
    perc_loss_donors=perc_loss_donors(history, filters_outliers_obs), 
    perc_loss_obs=perc_loss_obs(history, filters_outliers_obs)
)

outliers.df


# define filters

filters_outliers = (
    filter_rdate 
    & 
    filter_ramnt
)

# append to outliers overview
#append to oultier overview
outliers.append(
    var='total', 
    rule='whole donor with at least one outlier obs',
    perc_loss_donors=perc_loss_donors(history, filters_outliers), 
    perc_loss_obs=perc_loss_obs(history, filters_outliers)
)

outliers.df


In [ ]:
incons.df

In [ ]:
# apply filters
test_history_obs = history.loc[filters_outliers_obs,:] # scenario 'a'
test_history_donors = history.loc[filters_outliers,:] # scenario 'b'

perc_donors_incons = (1-history.CONTROLN.nunique() /donors.CONTROLN.nunique())*100
print(f'Donors lost due to inconsistencies in histroy data: {perc_donors_incons:,.2}%')

#perc_donors_ = history[CONTROLN_with_ramnt_outliers].CONTROLN.nunique() /donors.CONTROLN.nunique()
#print(f'{perc_donors_nohistory:,.2} % of the donors have no history at all!')

print('Additional loss due to outliers in history:')
total_perc_filterd_obs = (1-test_history_obs.CONTROLN.nunique() / history.CONTROLN.nunique())*100
print(f'- Perc donor loss szenario "a" (only dropping outlier obs) : {total_perc_filterd_obs:,.4}%')

total_perc_filterd_donors = (1-test_history_donors.CONTROLN.nunique() / history.CONTROLN.nunique())*100
print(f'- Perc donor loss szenario "b" (dropping whole doner with outliers): {total_perc_filterd_donors:,.4}%')



In [ ]:
filters_outliers

In [ ]:
donor_id_clean = history.loc[filters_outliers,:].CONTROLN.unique().tolist()
donor_id_out1 = history.loc[~filters_outliers,:].CONTROLN.unique().tolist()

len(donor_id_clean + donor_id_out1)

In [ ]:
test_history_obs.loc[test_history_obs.CONTROLN == CONTROLN_with_ramnt_outliers[0]]

In [ ]:
test_history_donors.loc[test_history_obs.CONTROLN == CONTROLN_with_ramnt_outliers[0]]

In [ ]:
# choose scenario b: 
history_out = history.loc[history.CONTROLN.isin(donor_id_out1),:]
history = test_history_donors


In [ ]:
history_out.CONTROLN.nunique()

# Feature Engineering 

In [ ]:
class ft(): 

    def month_diff_series(a, b):
        return 12 * (a.dt.year - b.dt.year) + (a.dt.month - b.dt.month)
    
    def month_diff(a,b): 
        return 12 * (a.year - b.year) + (a.month - b.month)


    def get_freq_days(df:pd.DataFrame, col): 
        s = (df.loc[:, col].dropna()).sort_values(ascending=False)
        #df = df.sort_values(by=col, ascending=False)

        res = (s.shift() - s).dropna().dt.days
        return(res)

    def get_freq_months(df:pd.DataFrame, col): 
        s = (df.loc[:, col].dropna()).sort_values(ascending=False)
        #df = df.sort_values(by=col, ascending=False)

        res = ft.month_diff_series(s.shift(), s).dropna()
        return(res)

    
    def get_days_to_last_donation(df, col, max_date): 
        res = (max_date - df.loc[:,col].max()).days
        return(res)

    def get_months_to_last_donation(df, col, max_date):
        #res = ft.month_diff(pd.Series(max_date), pd.Series(df.loc[:,col].max()))
        res = ft.month_diff(max_date, df.loc[:,col].max())
        return(res)
    
    def calc_reply_rate(x, col, ref): 
        res = (~x.loc[:,col].isna()).sum() / (~x.loc[:,ref].isna()).sum()
        res = round(res, 2)
        return(res)
    
    def get_rcy(x, date_col, lim_date): 
        res = (x[date_col] >= lim_date).sum()
        return(res)
    
    def pct_change(df, val_col, date_col): 
        # sort values to date
        df = df.sort_values(date_col, ascending=False)
        # calc percentage change, ignoring na values in between
        res = df.loc[~df[val_col].isna(), val_col].pct_change(-1)
        if len(res) == 0: 
            res = pd.Series([np.nan])
        return res
    
    def abs_change(df, val_col, date_col): 
        # sort values to date
        df = df.sort_values(date_col, ascending=False)
        # calc percentage change, ignoring na values in between
        res = df.loc[~df[val_col].isna(), val_col].diff(-1)
        if len(res) == 0: 
            res = pd.Series([np.nan])
        return res

#coeff variation: 
#https://statisticsbyjim.com/basics/coefficient-variation/

    def agg_fct(x, max_date): 
        d = {}
        d['donations_total'] = x['ramnt'].sum()
        d['donations_mean'] = x['ramnt'].mean()
        d['donations_std'] = round(np.std(x['ramnt']), 2)
        #print(ft.pct_change(x, 'ramnt', 'rdate'))
        d['perc_change_last'] = round(ft.pct_change(x, 'ramnt', 'rdate').iloc[0], 3)
        d['abs_change_last'] = round(ft.abs_change(x, 'ramnt', 'rdate').iloc[0], 3)
        d['perc_change_mean'] = round(ft.pct_change(x, 'ramnt', 'rdate').mean(), 3)
        d['perc_change_sd'] = round(ft.pct_change(x, 'ramnt', 'rdate').std(), 3)
        d['n_donations'] = (~x.loc[:,'ramnt'].isna()).sum()
        d['freq_mean'] = ft.get_freq_months(x,'rdate').mean()
        #print(np.std(ft.get_freq_months(x,'rdate')))
        d['freq_std'] = round(np.std(ft.get_freq_months(x,'rdate')), 2)
        d['months_to_last_donation'] = ft.get_months_to_last_donation(x, 'rdate', max_date)
        d['perc_reply_rate'] =  ft.calc_reply_rate(x, 'rdate', 'adate')
        d['mean_reply_time_mean'] = round((x['rdate'] - x['adate']).dt.days.mean(), 2)
        d['mean_reply_time_std'] = round(np.std((x['rdate'] - x['adate']).dt.days), 2)
        d['rcy'] = ft.get_rcy(x, 'rdate', rcy_lim_date)
        return pd.Series(d)

max_rdate = history.rdate.max()
print('max_rdate', max_rdate)
max_date = max_rdate + DateOffset(months=13)
print('maxdate', max_date)
rcy_lim_date = max_date - DateOffset(months=13+6)
print('rcy_lim_date', rcy_lim_date)

## Explore frequency

In [ ]:
#history.set_index(['CONTROLN'], append=True, inplace=True)
#history.reset_index(level=1, inplace=True)

In [ ]:
#history.describe(include='all')

#rdate_freq = history1688.groupby('CONTROLN').rdate.transform(lambda x: (x.shift() - x).dropna().dt.days)


def get_freq_series(x:pd.Series): 
    x = x.sort_values(ascending=False)
    
    #res = (x.shift() - x).dt.days
    res = ft.month_diff_series(x.shift(), x)
    return(res)
    
rdate_freq = history.loc[~history.rdate.isna(),:].groupby('CONTROLN').rdate.transform(lambda x: get_freq_series(x))
rdate_freq
#(rdate_freq <=0).value_counts()

rdate_freq_df = rdate_freq.reset_index(level='CONTROLN').dropna()
rdate_freq_df.loc[rdate_freq_df.CONTROLN == 1688,:]

rdate_freq_clean = rdate_freq.dropna()
(rdate_freq_clean<0).sum()
(rdate_freq_clean==0).sum()

 ## Apply Feature engineering

In [ ]:
### apply feature engineering to doner 1688
check_ctrl_no=[1688]
history_agg = history.loc[history.CONTROLN.isin(check_ctrl_no),:].groupby('CONTROLN').progress_apply(ft.agg_fct, max_date=max_date)
history_agg

In [ ]:
history[history.CONTROLN == 1688]

In [ ]:
### apply feature engineering to sample dataset
check_ctrl_no = random.choices(population=history.CONTROLN.unique(), k=150)
history_agg = history.loc[history.CONTROLN.isin(check_ctrl_no),:].groupby('CONTROLN').progress_apply(ft.agg_fct, max_date=max_date)
history_agg
#check

In [ ]:
### apply feature engineering to complete dataset
    
if 'history_feat_raw.pickle' in os.listdir(computed_data_path): 
    with open(os.path.join(computed_data_path, 'history_feat_raw.pickle'), 'rb') as f: 
        history_agg = pickle.load(f)
        
else: 
    history_agg = history.groupby('CONTROLN').progress_apply(ft.agg_fct, max_date=max_date)

    with open(os.path.join(computed_data_path, 'history_feat_raw.pickle'), 'wb') as f: 
        pickle.dump(history_agg, f)

history_agg
    
    

In [ ]:
history_out

In [ ]:
### apply feature engineering to outlier dataset
    
if 'history_feat_raw_out.pickle' in os.listdir(computed_data_path): 
    with open(os.path.join(computed_data_path, 'history_feat_raw_out.pickle'), 'rb') as f: 
        history_agg_out = pickle.load(f)
        
else: 
    history_agg_out = history_out.groupby('CONTROLN').progress_apply(ft.agg_fct, max_date=max_date)

    with open(os.path.join(computed_data_path, 'history_feat_raw_out.pickle'), 'wb') as f: 
        pickle.dump(history_agg_out, f)

    history_agg_out
    

In [ ]:
history_out.CONTROLN.nunique()
history_agg_out
history_agg_out.shape[0] / history_agg.shape[0]

## Step2 Features

In [ ]:
loc = max(history_agg.columns.get_loc('freq_std'), history_agg.columns.get_loc('freq_mean'))
loc
history_agg.insert(
    loc=loc+1,
    column='stability_idx',
    value=history_agg.freq_std / history_agg.freq_mean
)


loc = max(history_agg.columns.get_loc('perc_change_sd'), history_agg.columns.get_loc('perc_change_sd'))
loc
history_agg.insert(
    loc=loc+1,
    column='value_stability_idx',
    value=history_agg.perc_change_sd / history_agg.perc_change_mean
)


history_agg

In [ ]:
history[history.CONTROLN == 41]

## Explorations on features

In [ ]:
# check na in data with at least 2 donations (smaller than that NAs ok to exsist by definition)
na_onetime = history_agg.loc[history_agg.n_donations ==1,:].isna().sum()
na_twotime = history_agg.loc[history_agg.n_donations ==2,:].isna().sum()
na_multiple = history_agg.loc[history_agg.n_donations >=3,:].isna().sum()
na_never = history_agg.loc[history_agg.n_donations ==0,:].isna().sum()
na_total = history_agg.isna().sum()

if False: 
    print(f'Total NAs:\n{history_agg.isna().sum()}')
    print(f'\nNAs single:\n{na_onetime}')
    print(f'\nNAs multi2:\n{na_twotime}')
    print(f'\nNAs multi>2:\n{na_multiple}')

na_donations = pd.DataFrame(
    [na_never, na_onetime, na_twotime, na_multiple],
    index=['never', 'onetime', 'twotime', 'multiple']
).T

if not all(na_donations.sum(axis=1) == na_total): 
    raise Warning('upps')

na_donations.to_excel(os.path.join(explorations_data_path, 'na_hist_feat.xlsx'))

In [ ]:
25057 / history_agg.shape[0]
history_agg.shape[0]

In [ ]:
history_agg.iloc[977,:]
history[history.CONTROLN == 977]

In [ ]:
# donors with sd in percentage change is NA
# exclusively donors with 3 donations -> by design, but what to do?
multidonors_with_percchange_errors = history_agg[(history_agg.perc_change_sd.isna()) & (history_agg.n_donations >=2)].index.to_list()
history[history.CONTROLN.isin(multidonors_with_percchange_errors[0:1])]


In [ ]:
# Donors with percentage change > 1
# ok, the more than doubled...
donors_high_percchange = history_agg[history_agg.perc_change_mean > 1].index.to_list()
history[history.CONTROLN.isin(donors_high_percchange[0:1])]


In [ ]:
# donors with no histroy 
CONTROLN_nohist = history_agg.loc[history_agg.n_donations == 0,:].index.to_list()
print(f'Number of donors with no donation within histroy: {(len(CONTROLN_nohist) / history_agg.shape[0]): .2%}')
#donors.loc[donors.CONTROLN.isin(CONTROLN_nohist), cols_with_hist]

# one time donors  
CONTROLN_onetime = history_agg.loc[history_agg.n_donations == 1,:].index.to_list()
print(f'Number of onetime donors: {(len(CONTROLN_onetime) / history_agg.shape[0]): .2%}')
#donors.loc[donors.CONTROLN.isin(CONTROLN_onetime), cols_with_hist]

CONTROLN_multi = history_agg.loc[history_agg.n_donations >= 2,:].index.to_list()

apriory = pd.DataFrame({
    'nDonors': [len(CONTROLN_nohist),len(CONTROLN_onetime),len(CONTROLN_multi) ]
}, index=['never', 'onetime', 'multiple'])


sns.barplot(data=apriory, x=apriory.index, y='nDonors')

plt.show()

In [ ]:
# Prepare figure
fig = plt.figure(figsize=(10, 8))

# Obtain correlation matrix. Round the values to 2 decimal cases. Use the DataFrame corr() and round() method.
corr = np.round(history_agg.corr(method="pearson"), decimals=2)

# Build annotation matrix (values above |0.5| will appear annotated in the plot)
mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) # Try to understand what this np.where() does

# Plot heatmap of the correlation matrix
sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

# Layout
fig.subplots_adjust(top=0.95)
#fig.suptitle("Correlation Matrix", fontsize=20)

plt.tight_layout()

plt.savefig(os.path.join(explorations_data_path, 'correlation_matrix.png'), dpi=200)
plt.show()

In [ ]:
if False: 
    # Pairwise Relationship of Numerical Variables
    sns.set()

    # Setting pairplot
    sns.pairplot(history_agg.iloc[:,:], diag_kind="none", corner=False)

    # Layout
    plt.subplots_adjust(top=0.95)
    plt.suptitle("Pairwise Relationship of Numerical Variables", fontsize=20)

    plt.savefig(os.path.join(explorations_data_path, 'pairwise_relationship_of_numerical_variables.png'), dpi=200)

    plt.show()


### Ideas
- use n donations only to define n=1, n=2 a priori
- drop rcy and take month to last donatio instead
    - highly corr 
    - rcy only has few unique values

In [ ]:
feats_to_drop = ['value_stability_idx', 'rcy', 'perc_change_sd', 'freq_std']

In [ ]:
current_palette = sns.color_palette()
sns.palplot(current_palette)

In [ ]:
def get_donation_gr(history_agg): 
    hue_l = ['multiple' if d >=3 else 'twotime' if d >=2 else 'onetime' if d != 0 else 'never' for d in history_agg.n_donations]
    hue_order_l = ['never', 'onetime','twotime','multiple']
    return hue_l, hue_order_l

hue_l, hue_order_l = get_donation_gr(history_agg.drop(columns=feats_to_drop))
#palette = dict(zip(hue_order_l, current_palette[0:4]))
palette_n_donations = {
    'never':current_palette[3] ,
    'onetime': current_palette[2],
    'twotime': current_palette[1],
    'multiple': current_palette[0]
} 
fig = plt.figure()
sns.countplot(x = hue_l, palette=palette_n_donations)
plt.savefig(os.path.join(explorations_data_path, 'nDonations_grouped.png'), dpi=200)
plt.show()


In [ ]:
def plot_distribution(history_agg, palette, name): 
    ncols = 4
    n_plots = history_agg.shape[1]
    nrows = int(np.ceil(n_plots/ncols))


    if palette is not None:
        hue_l, hue_order_l = get_donation_gr(history_agg)
    else: 
        hue_l, hue_order_l = (None, None)

    legend = [False]*(n_plots)
    legend[0] = True              

    fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15,10))
    col_no = 0
    for i in range(nrows):
        for j in range(ncols): 
            if col_no < n_plots:
                col = history_agg.columns[col_no]
                leg = legend[col_no]
                print(col)
                sns.histplot(
                    legend=leg, data=history_agg, x=col,
                    hue=hue_l, hue_order=hue_order_l,
                    ax=ax[i,j], bins=30,
                    palette=palette, alpha=.7
                ).set_title(col)
                col_no +=1
    fig.tight_layout()

    plt.savefig(os.path.join(explorations_data_path, f'distributions_{name}.png'), dpi=200)
    plt.show()

plot_distribution(history_agg.drop(columns=feats_to_drop), palette=palette_n_donations, name='raw')


In [ ]:
history_agg.shape

#  Feature cleaning 

### stability_idx

In [ ]:
def get_range(x:list): 
    if type(x) is list: 
        x = pd.Series(x)
    return(x.min(), x.max())

# multidoners with missing stability_idx
#print(range(history_agg.stability_idx.values))
history_agg[
    (history_agg.n_donations > 1) 
    & (history_agg.stability_idx.isna())
    & (history_agg.freq_mean == 0)
]


print('range_stability_idx: ', get_range(history_agg.stability_idx))

filter_multi_stabidx_missing = (
    (history_agg.stability_idx.isna())
    & (history_agg.n_donations >=2)
)
    
CONTROLN_to_check = history_agg[filter_multi_stabidx_missing].index.to_list()[0:1]
history[history.CONTROLN.isin(CONTROLN_to_check)].sort_values(['CONTROLN', 'adate'], ascending=False)

# impute
history_agg.loc[filter_multi_stabidx_missing, 'stability_idx'] = 0

history_agg[history_agg.n_donations >=3].stability_idx.isna().sum()


### reply_time

In [ ]:
# multi donors but still na in mean reply time...
filter_multi_reply_time_na = (
    (history_agg.n_donations >=2) 
    & (history_agg.mean_reply_time_mean.isna())
)
check_no = history_agg.loc[filter_multi_reply_time_na,:].index.to_list()
check_no
history.loc[history.CONTROLN.isin(check_no),:]
#donors.loc[donors.CONTROLN.isin(check_no),cols_with_hist]

# drop case
history_agg = history_agg[~filter_multi_reply_time_na]
history_agg[history_agg.n_donations >=3].mean_reply_time_mean.isna().sum()

history_agg.shape

# Outliers removal on features

In [ ]:
if True: 
    feats_to_drop += [
        'n_donations'
    ]

feats_to_drop

In [ ]:
cols_with_feats = history_agg.drop(columns=feats_to_drop).columns.to_list()

In [ ]:
filter_multi = history_agg.n_donations >=2
history_agg_multi = history_agg[filter_multi].drop(columns=feats_to_drop)
history_agg_multi.shape

In [ ]:
def rm_outlier_manual(df, col, th): 
    

    if th is None: 
        th = df[col].max()
    filter_col = df[col] <= th
    sns.histplot(data=df[filter_col], x=col, bins=30)

    perc_loss = round((1 - filter_col.sum() / df.shape[0])*100, 2)
    print(f'perc_loss: {perc_loss}%')
    plt.show()

    return col, th, perc_loss, filter_col

outliers_manual = {}
filter_out_man = {}



In [ ]:
col, th, perc_loss, filter_col = rm_outlier_manual(
    df=history_agg_multi,
    col='perc_change_last',
    th=3
)

outliers_manual[col] = (th, perc_loss)
filter_out_man[col] = filter_col


In [ ]:
col, th, perc_loss, filter_col = rm_outlier_manual(
    df=history_agg_multi,
    col='perc_change_mean',
    th=3
)

outliers_manual[col] = (th, perc_loss)
filter_out_man[col] = filter_col

In [ ]:
col, th, perc_loss, filter_col = rm_outlier_manual(
    df=history_agg_multi,
    col='mean_reply_time_std',
    th=120
)

outliers_manual[col] = (th, perc_loss)
filter_out_man[col] = filter_col

In [ ]:
col, th, perc_loss, filter_col = rm_outlier_manual(
    df=history_agg_multi,
    col='stability_idx',
    th=1.3
)

outliers_manual[col] = (th, perc_loss)
filter_out_man[col] = filter_col

In [ ]:
if 'n_donations' in history_agg_multi.columns.to_list():
    col, th, perc_loss, filter_col = rm_outlier_manual(
        df=history_agg_multi,
        col='n_donations',
        th=12
    )

    outliers_manual[col] = (th, perc_loss)
    filter_out_man[col] = filter_col

In [ ]:
loss_df_man = pd.DataFrame(outliers_manual, index=['value', 'perc_loss']).T
loss_df_man.insert(
    loc=0, 
    column='rule', 
    value='manual'
)

loss_df_man


In [ ]:
# combine all manual filters

filter_out_man_df = pd.DataFrame(filter_out_man)
filter_out_man_df

filter_out_man_comb = np.all(filter_out_man_df, 1)
filter_out_man_comb

# 
feats_out_man = filter_out_man_df.columns.to_list()
feats_out_man 


In [ ]:
filter_outliers_init = []
perc_loss_init = []
for col in history_agg_multi.drop(columns=feats_out_man).columns: 
    
    filter_tmp = def_outliers_iqr(history_agg_multi[col], factor=1.5, omit_na=False)
    filter_outliers_init.append(filter_tmp)
    
    loss = (1-(filter_tmp.sum() / history_agg_multi.shape[0]))*100
    perc_loss_init.append((col, loss))
    if False: 
        print(col)
        fig, ax = plt.subplots(ncols=2)
        sns.histplot(data=history_agg_multi, x = col, bins=30, ax=ax[0])
        sns.histplot(data=history_agg_multi[filter_tmp], x = col, bins=30, ax=ax[1])
        plt.show()

perc_loss_init
#filter_outliers

In [ ]:
loss_df = pd.DataFrame(perc_loss_init, columns=['col', 'perc_loss_init']).sort_values('perc_loss_init')
loss_df

In [ ]:
loss_df['factor'] = [1.5 if l<1 else 2.5 if l<5 else 5 if l<10 else 6.5 for l in loss_df.perc_loss_init.values]
if 'col' in loss_df.columns: 
    loss_df.set_index('col', inplace=True)
    
loss_df.loc['donations_mean','factor'] = 3.5
#loss_df.loc['n_donations','factor'] = 3.5
#loss_df.loc['perc_change_last','factor'] = 10
#loss_df.loc['perc_change_mean','factor'] = 10
#loss_df.loc['mean_reply_time_std','factor'] = 7.5

loss_df

In [ ]:
filter_outliers = []
perc_loss = []
for col in history_agg_multi.drop(columns=feats_out_man).columns: 
    fac = loss_df.loc[col,:].factor
    print(col, fac)

    
    filter_tmp = def_outliers_iqr(history_agg_multi[col], factor=fac, omit_na=False)
    filter_outliers.append(filter_tmp)
    
    loss = round((1-(filter_tmp.sum() / history_agg_multi.shape[0]))*100, 4)
    perc_loss.append((col, loss))
    if False: 
        fig, ax = plt.subplots(ncols=2)
        sns.histplot(data=history_agg_multi, x = col, bins=30, ax=ax[0])
        sns.histplot(data=history_agg_multi[filter_tmp], x = col, bins=30, ax=ax[1])
        plt.show()

perc_loss
#filter_outliers

In [ ]:
tmp = pd.DataFrame(perc_loss, columns=['col', 'perc_loss']).sort_values('perc_loss')
tmp.set_index('col', inplace=True)
loss_df = pd.concat([loss_df, tmp], axis=1)#.sort_values('perc_loss')


In [ ]:
filters2 = pd.Series(np.all(filter_outliers, 0), index=history_agg_multi.index)

filters2_comb = (
    filters2
    & filter_out_man_comb
)
filters2_comb

history_agg_multi_clean = history_agg_multi[filters2_comb].copy()

perc_loss_outliers2 = (1- history_agg_multi_clean.shape[0] / history_agg_multi.shape[0])*100
print(f'perc_loss_outliers2: {perc_loss_outliers2:.4}%')



In [ ]:
loss_df.sort_values('factor', inplace=True)
loss_df['perc_loss_init'] = round(loss_df['perc_loss_init'],4)

loss_df_iqr = loss_df.copy()
loss_df_iqr.rename(columns={'factor': 'value'}, inplace=True)
loss_df_iqr.insert(
    loc=1,
    column='rule', 
    value='iqr - factor'
)

loss_df_comb = pd.concat([loss_df_man, loss_df_iqr])
loss_df_comb


In [ ]:

loss_df_comb.loc['sum'] = np.nan
loss_df_comb.loc['sum', 'perc_loss'] = round(loss_df_comb.perc_loss.sum(), 2)

loss_df_comb.loc['overall'] = np.nan
loss_df_comb.loc['overall', 'perc_loss'] = round(perc_loss_outliers2, 2)


loss_df_comb.to_excel(os.path.join(explorations_data_path, 'outliers2.xlsx'))

loss_df_comb


In [ ]:
plot_distribution(history_agg_multi_clean, palette=None, name='clean')


In [ ]:
# drop features
history_agg_multi_clean

In [ ]:
filters2_comb[filters2_comb == False]

In [ ]:
history_agg_out

In [ ]:
donor_id_out2 = filters2_comb[filters2_comb == False].index.to_list()
donor_id_out2

history_agg_out2 = history_agg[history_agg.index.isin(donor_id_out2)]

history_agg_out_total = pd.concat([history_agg_out, history_agg_out2])
history_agg_out_total

In [ ]:
history_agg_out_total.shape[0] / history_agg.shape[0]

In [ ]:
history_agg_out_total.n_donations.value_counts()

In [ ]:
history_agg_out.n_donations.value_counts()


In [ ]:
history_agg_out_multi = history_agg_out.loc[history_agg_out.n_donations >=2,:]
history_agg_out_multi

In [ ]:
# save results

with open(os.path.join(computed_data_path, 'history_feat_multi_out.pickle'), 'wb') as f: 
    pickle.dump(history_agg_out_multi, f)
    
with open(os.path.join(computed_data_path, 'history_feat_multi_clean.pickle'), 'wb') as f: 
    pickle.dump(history_agg_multi_clean, f)
        

## Feature profile report
profile = ProfileReport(
    history_agg, 
    title='Donors Hist Agg1',
    minimal=False, 
    correlations={
    "pearson": {"calculate": True},
    "spearman": {"calculate": False},
    "kendall": {"calculate": False},
    "phi_k": {"calculate": False},
    "cramers": {"calculate": False},
    }
)
profile.to_file('explorations/profile_hist_agg1_v2.html')

profile = ProfileReport(
    history_agg_2,
    title='Donors Hist Agg 2',
    minimal=False, 
    correlations={
    "pearson": {"calculate": True},
    "spearman": {"calculate": False},
    "kendall": {"calculate": False},
    "phi_k": {"calculate": False},
    "cramers": {"calculate": False},
    }
)
profile.to_file('explorations/profile_hist_agg_2.html')

In [ ]:
cols_with_histsumm = [
    'RAMNTALL',
    'NGIFTALL',
    'CARDGIFT',
    'MINRAMNT',
    'MINRDATE',
    'MAXRAMNT',
    'MAXRDATE',
    'LASTGIFT',
    'LASTDATE',
    'FISTDATE',
    'NEXTDATE',
    'TIMELAG',
    'AVGGIFT'
]

donors.loc[:,cols_with_histsumm + ['CONTROLN']].set_index('CONTROLN', append=True)

profile = ProfileReport(
    donors.loc[:,cols_with_histsumm], 
    title='Donors Hist Summary',
    minimal=False, 
    correlations={
    "pearson": {"calculate": True},
    "spearman": {"calculate": False},
    "kendall": {"calculate": False},
    "phi_k": {"calculate": False},
    "cramers": {"calculate": False},
    }
)
profile.to_file('explorations/profile_hist_summ.html')